In [14]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point
import leafmap.foliumap as leafmap

In [15]:
# Directory
current = Path.cwd()
parent = current.resolve().parent / 'data'
os.chdir(parent)
os.getcwd()

'C:\\Users\\alons\\OneDrive\\Documents\\Code\\Python\\CondaSpatial\\Streamlit\\LoisaidaOpenDataHub\\data'

In [16]:
# Load data
NTA20 = gpd.read_file("2020 Neighborhood Tabulation Areas (NTAs).geojson")
Pluto = pd.read_csv("pluto_24v2.csv")
LL104 = pd.read_csv("LL104_properties.csv")
Boba = pd.read_csv("bobaadr.csv")

In [17]:
# Processing
NTA20_MN = NTA20[NTA20['boroname'] == 'Manhattan']
PlutoMN = Pluto[Pluto['borough'] == 'MN']
LL104MN = LL104[LL104['BOROUGH'] == 'Manhattan']
Boba = Boba[Boba['boro'] == 1]

Boba = Boba.drop_duplicates(subset='bin')

JoinTest = pd.merge(LL104MN, Boba, how='left', left_on='BIN', right_on='bin')

JoinTest['boro'] = '10'
JoinTest['block'] = JoinTest['block'].astype(str).str.zfill(4)
JoinTest['lot'] = JoinTest['lot'].astype(str).str.zfill(4)
JoinTest['bbl'] = JoinTest['boro'] + JoinTest['block'] + JoinTest['lot']
JoinTest = JoinTest.drop_duplicates(subset='bbl')

PlutoMN['bbl'] = PlutoMN['bbl'].astype(int).astype(str)

JoinTestPluto = pd.merge(JoinTest, PlutoMN, how='left', on ='bbl')

JoinTestPluto = JoinTestPluto[['ADDRESS', 'BIN', 'HPD_VIOLATIONS', 'DOB_VIOLATIONS', 'TOTAL_VIOLATIONS', 'DWELLING_UNITS', 'VIO_UNITS_RATIO',
                               'latitude', 'longitude']]

geometry = [Point(xy) for xy in zip(JoinTestPluto['longitude'], JoinTestPluto['latitude'])]
crs = {'init': 'epsg:4326'}
Viol_gdf = gpd.GeoDataFrame(JoinTestPluto, crs=crs, geometry=geometry)


In [18]:
# Returning data
NTA20_MN.to_file("NTA20_MN.geojson", driver='GeoJSON')
Viol_gdf.to_file('LES_Buildings_OpenViolations.geojson', driver='GeoJSON')  

In [19]:
# Plotting
m = leafmap.Map(center=[40.7248387,-73.9775537], zoom=12)
m.add_gdf(gdf=NTA20_MN, layer_name='Manhattan Neighborhood Tabulation Areas (NTAs)', info_mode='on_click')
#m.add_labels(data=NTA20_MN, column='ntaname', font_size='12pt', font_color='black', font_family='arial', font_weight='normal')
m.add_gdf(gdf=Viol_gdf, layer='Open DOB Violations', info_mode= 'on_hover', zoom_to_layer=True)
#m.add_layer_control(position='bottomright')
#m.add_layer_manager(position='topright')